In [1]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [2]:
# # 데몬 인스턴스 만들기 
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

In [3]:
es_username = 'elastic'
es_password = 'tYlfsjnZ_OGLcM81_aU2'

# ElasticSearch Client 생성 
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/upstage-ai-final-ir2/HM/elasticsearch-8.8.0/config/certs/http_ca.crt", verify_certs=False)
# CA 인증서 경로 지정

# ElasticSearch Client 정보 확인 
es.info()

/root/home/envforir/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'instance-8909', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'r4Ih9u1sTfKbhPDz61kqLQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
# 데몬 구동 확인 
!ps -ef | grep elasticsearch

iruser    896724       1 99 23:25 pts/16   00:00:53 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-1671164173185419744 -XX:+HeapDumpOnOutOfMemoryError -XX:+ExitOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,level,pid,tags:filecount=32,filesize=64m -Xms30720m -Xmx30720m -XX:MaxDirectMemorySize=16106127360 -XX:InitiatingHeapOccupancyPercent=30 

# 검색함수

In [7]:
# 역색인을 이용한 검색 
def sparse_retrieve(query_str, size):
    query = { "match": {"content": {"query": query_str}}}
    return es.search(index='test', query=query, size=size, sort='_score')
#>> query_str: 검색어
#>> size: 검색 결과 개수 
#>> 'test' 인덱스에서 정의된 쿼리를 실행 -> 검색 결과를 size개 만큼 반환 

In [18]:
# dense_retrieve_2: sparse_retrieve 함수로 필터링된 문서들에 대해서만 유사도 검색
# dense_retrieve_2: sparse_retrieve 함수로 필터링된 문서들에 대해서만 유사도 검색
def dense_retrieve_2(query_str, documents, size):
    query_embedding = get_embedding(query_str)  # 리스트로 감싸지 않음
    document_ids = [doc['_id'] for doc in documents]
    
    # docid와 _id의 매핑 생성
    id_to_docid = {doc['_id']: doc['_source']['docid'] for doc in documents}

    body = {
        "_source": {"includes": ["content", "embeddings"]},
        "query": {
            "bool": {
                "must": [
                    {
                        "ids": {
                            "values": document_ids
                        }
                    },
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": """
                                    double euclidean_distance = 0;
                                    for (int i = 0; i < params.query_vector.length; i++) {
                                        euclidean_distance += Math.pow(params.query_vector[i] - doc['embeddings'][i], 2);
                                    }
                                    euclidean_distance = Math.sqrt(euclidean_distance);
                                    return 1.0 / (1.0 + euclidean_distance);
                                """,
                                "params": {"query_vector": query_embedding}
                            }
                        }
                    }
                ]
            }
        }
    }

    results = es.search(index='test', body=body, size=size)
    
    # 결과를 스코어에 따라 정렬하고 docid 추가
    sorted_results = sorted(results['hits']['hits'], key=lambda x: x['_score'], reverse=True)
    for result in sorted_results:
        result['_source']['docid'] = id_to_docid[result['_id']]
    
    return {'hits': {'hits': sorted_results[:size]}}


In [9]:
import json
from elasticsearch import Elasticsearch, helpers
from openai import OpenAI
from dotenv import load_dotenv
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Elasticsearch 클라이언트 초기화
# es = Elasticsearch(['http://localhost:9200'])  # 필요에 따라 URL 변경

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# OpenAI API를 사용하여 임베딩 생성
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):
    if isinstance(text, list):
        text = " ".join(text)  # 리스트인 경우 문자열로 결합
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i: i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = [get_embedding(content) for content in contents]
        batch_embeddings.extend(embeddings)
        print(f'Processed batch {i}')
    return batch_embeddings

# 인덱스 생성 함수
def create_es_index(index, settings, mappings):
    if es.indices.exists(index=index):
        es.indices.delete(index=index)
    es.indices.create(index=index, settings=settings, mappings=mappings)
    print(f"Index '{index}' created successfully.")

# bulk_add 함수
def bulk_add(index, docs):
    actions = [{'_index': index, '_source': doc} for doc in docs]
    try:
        success, failed = helpers.bulk(es, actions, stats_only=True, raise_on_error=False)
        print(f"Indexed {success} documents successfully.")
        if failed:
            print(f"Failed to index {failed} documents.")
            # 실패한 문서에 대한 자세한 정보를 얻기 위해 stats_only=False로 다시 실행
            _, errors = helpers.bulk(es, actions, stats_only=False, raise_on_error=False)
            for error in errors:
                print(f"Error: {error}")
        return success, failed
    except Exception as e:
        print(f"An error occurred during bulk indexing: {str(e)}")
        return 0, len(docs)

# 인덱스 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    },
    "index": {
        "mapping": {
            "total_fields": {
                "limit": 2000
            }
        },
        "max_docvalue_fields_search": 200
    }
}

# 매핑 설정
mappings = {
    "properties": {
        "content": {
            "type": "text",
            "analyzer": "nori"
        },
        "embeddings": {
            "type": "dense_vector",
            "dims": 1536,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}

# 인덱스 생성
create_es_index("test", settings, mappings)

# 문서 로드 및 임베딩 생성
index_docs = []
with open("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
success, failed = bulk_add("test", index_docs)

# 색인 결과 출력
print(f"Successfully indexed: {success}")
print(f"Failed to index: {failed}")

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Index 'test' created successfully.
Processed batch 0
Processed batch 100
Processed batch 200
Processed batch 300
Processed batch 400
Processed batch 500
Processed batch 600
Processed batch 700
Processed batch 800
Processed batch 900
Processed batch 1000
Processed batch 1100
Processed batch 1200
Processed batch 1300
Processed batch 1400
Processed batch 1500
Processed batch 1600
Processed batch 1700
Processed batch 1800
Processed batch 1900
Processed batch 2000
Processed batch 2100
Processed batch 2200
Processed batch 2300
Processed batch 2400
Processed batch 2500
Processed batch 2600
Processed batch 2700
Processed batch 2800
Processed batch 2900
Processed batch 3000
Processed batch 3100
Processed batch 3200
Processed batch 3300
Processed batch 3400
Processed batch 3500
Processed batch 3600
Processed batch 3700
Processed batch 3800
Processed batch 3900
Processed batch 4000
Processed batch 4100
Processed batch 4200


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/c

Indexed 4272 documents successfully.
Successfully indexed: 4272
Failed to index: 0


In [10]:
# 임베딩을 파일로 저장 
import json

def save_embeddings_to_json(docs, embeddings, filename):
    data = []
    for doc, embedding in zip(docs, embeddings):
        doc_with_embedding = doc.copy()
        doc_with_embedding['embedding'] = embedding
        data.append(doc_with_embedding)
    
    with open(filename, 'w') as f:
        json.dump(data, f)

# 사용 예:
save_embeddings_to_json(docs, embeddings, 'embeddings.json')

In [11]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "직류와 교류 전류의 차이에 대해 알려줘"

In [12]:
sparse_results = sparse_retrieve(test_query, 30)

print("Sparse retrieve results:")
for rst in sparse_results['hits']['hits']:
    print('ID:', rst['_source']['docid'], 'score:', rst['_score'], 'source:', rst['_source']['content'][:100])

Sparse retrieve results:
ID: 144f5e5e-8069-425f-80b3-6388195ba4ee score: 25.206154 source: 전기 회로에서 DC와 AC의 차이점은 다음과 같다. DC는 직류 전류를 의미하며, 전하가 한 방향으로 흐른다는 것이 특징이다. 이는 전류의 방향이 일정하게 유지되는 것을 의미한다.
ID: 785aec2e-3be5-4c54-bc2a-68714ab88d30 score: 13.403629 source: 직류 전압을 측정하는 데에는 여러 가지 방법이 있습니다. 그 중에서도 전위차계 방법은 전압계를 사용한 직접 측정보다 더 정확한 결과를 얻을 수 있습니다. 이는 전위차계 방법이 회로
ID: 7b086212-c74b-413f-b414-d5d893f59152 score: 12.6978855 source: 네 개의 동일한 교류 발전기가 병렬로 작동하고 있으며, 각각의 등급은 20 MVA이고, 전압은 11 KV입니다. 이 발전기들은 초기 과도 리액턴스가 16%로 설정되어 있습니다. 이
ID: 77f168a3-5ff4-44bb-98a0-ac8fce381173 score: 11.086004 source: 특정 네트워크의 모든 요소가 선형인 경우, 중첩원리가 유지되려면 여자가 AC 또는 DC이어야 한다.

선형 네트워크는 전기 회로에서 매우 중요한 개념입니다. 선형 네트워크는 모든 
ID: d74dccae-120f-40cd-87fb-78b61ac97e42 score: 11.01634 source: 미국에서는 남부와 나머지 지역 사이에서 학교 인종차별 철폐에 대한 지지도 차이가 존재했습니다. 이러한 차이는 대략 40% 정도였습니다. 남부 지역에서는 학교 인종차별 철폐에 대한 
ID: a2384d59-df86-4cd0-844d-968295c6a0e7 score: 10.741834 source: 2극 중권 직류기에서 도체 한 개의 저항이 2 Ω이고 도체의 총수는 100이다. 따라서, 총 저항은 50Ω입니다. 이는 도체의 저항이

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
# dense_retrieve_2
dense_results = dense_retrieve_2(test_query, sparse_results['hits']['hits'], 3)


print("\nDense retrieve 2 results:")
for rst in dense_results['hits']['hits']:
    print('ID:', rst['_source']['docid'], 'score:', rst['_score'], 'source:', rst['_source']["content"][:100])

/tmp/ipykernel_896450/3656039920.py:41: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index='test', body=body, size=size)
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')